In [194]:
import numpy as np
from scipy.integrate import quad


def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)
    
R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618 # bar·m³/(K·mol)
Tpad = 298.15 # temperatura padrão 25 Celcius

class Comp: # Compostos : Classe que abriga as propriedades e funções
    def __init__(self, omega, Pc, Tb, Tc, NCParams, Ant1 = None, Ant2 = None, Ant3 = None): # NCParams: A, B, C, D cp do V.Ness; Ant: A, B, C 
        self.omega = omega
        self.Pc = Pc
        self.Tb = Tb
        self.Tc = Tc
        self.NCParams = NCParams
        self.Ant1 = Ant1
        self.Ant2 = Ant2
        self.Ant3 = Ant3
    
    def Ant(self, T): # faixas de antoine, Escolhe uma
        if T > self.Ant1[3] and T < self.Ant1[4]:
            return self.Ant1[:3]

        if T > self.Ant2[3] and T < self.Ant2[4]:
            return self.Ant2[:3]

        if T > self.Ant3[3] and T < self.Ant3[4]:
            return self.Ant3[:3]

        return self.Ant1

    def Antoine(self, T): # NIST, T em K
        Ant = self.Ant(T)
        AntA = Ant[0]
        AntB = Ant[1]
        AntC = Ant[2]
        return 10 ** (AntA - (AntB / (T + AntC)))

    def TAntoine(self, P): # Eq antoine para temperatura
        Ant = self.Ant1
        T = (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]
        if self.Ant3 == None:
            return T
        Ant = self.Ant(T) # roda novamente para escolher a faixa correta de antoine
        return (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]

    def Estado(self, P, T): # Retorna o estado do composto
        if T > self.Tc and P > self.Pc:
            return "gas"
        elif T > self.Tc:
            return "gas"
        elif self.Ant1 == None:
            return "gas"
        elif T < self.Tc and P > self.Pc:
            return "liquid"
        elif T < self.Tc and P < self.Pc:
            if  P < self.Antoine(T):
                return "vapour"
            else:
                return "liquid"

    def Cp_ig(self, T): # Equação 142 V.Ness
        A = self.NCParams[0]
        B =  self.NCParams[1]
        C = self.NCParams[2]
        D =  self.NCParams[3]
        return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))
    
    def Cp_L(self, T): # Cp de Rowlison Bondi
        return R * (1.45 + (0.45/(1-(T / self.Tc)) + 0.25 * self.omega * (17.11 + (25.2 * (1 - (T / self.Tc))**(1/3)) / (T / self.Tc) + (1.742)/(1 - (T / self.Tc)) ) ) ) + self.Cp_ig(T)
    
    def Hres(self, P, T): # Por Peng Robinson slide aula 2 integração
        P = toPa(P)
        k = 0.37464 + (0.154226 * self.omega) - (0.26992 * (self.omega ** 2))
        alphat = (1 + k * (1 - np.sqrt(T/self.Tc))) ** 2
        ac = 0.45724 * (((R ** 2) * (self.Tc ** 2)) / toPa(self.Pc))
        a = ac * alphat
        b = 0.07780 * R * (self.Tc / toPa(self.Pc))
        A = (a * P) / ((R ** 2) * (T ** 2.5))
        B =  (b * P) / (R * T)
        alpha = -1 + B
        beta = A - (3 * B ** 2 ) - 2 * B
        gamma = - (A * B) + (B ** 2) + (B ** 3)
        Z = np.roots([1, alpha, beta, gamma])
        Z = Z[np.isreal(Z)]
        Z = max(Z)
        Tdadt = - ac * k * alphat * (T / self.Tc)
        h = (b * P) / (Z * R * T)
        return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

    def Hvap(self): # Por Riedel p147 V.Ness
        return R * self.Tb * ((1.092 * (np.log(self.Pc) - 1.013)) / (0.930 - (self.Tb / self.Tc))) # P em bar mesmo

    def Hmolar(self, P, T): # Calcula a entalpia molar
        match self.Estado(P, Tpad): # estado inicial
            case "gas" | "vapour":
                match self.Estado(P, T):
                    case "gas" | "vapour":
                        # print("caminho gas - gas")
                        H = quad(self.Cp_ig, Tpad, T)[0] + self.Hres(P, T)
                        # print(f"{H} J/mol")
                        return H
                    case "liquid":
                        # print("caminho gas - liquido")
                        Tvap = self.TAntoine(P)
                        HGas = quad(self.Cp_ig, Tpad, Tvap)[0] + self.Hres(P, Tvap)
                        Hv = self.Hvap()
                        Hl = quad(self.Cp_L, Tvap, T)[0]
                        H = HGas - Hv + Hl # Menos Hvaporização já que é o processo inverso, condensação
                        # print(f"{H} J/mol")
                        return H
            case "liquid":
                match self.Estado(P, T):
                    case "gas" | "vapour":
                        # print("caminho liquido - gas")
                        Tvap = self.TAntoine(P)
                        # print(f"tvap = {Tvap}")
                        Hl = quad(self.Cp_L, Tpad, Tvap)[0]
                        # print(f"Hl = {Hl}")
                        Hv = self.Hvap()
                        # print(f"Hv = {Hv}")
                        HGas = quad(self.Cp_ig, Tvap, T)[0] + self.Hres(P, T)
                        # print(f"HGas = {HGas}")
                        H = Hl + Hv + HGas
                        # print(f"{H} J/mol\n")
                        return H
                    case "liquid":
                        # print("caminho liquido - liquido")
                        H = quad(self.Cp_L, Tpad, T)[0]
                        # print(f"{H} J/mol")
                        return H

H2 = Comp( -0.217, # Prausnitz
          12.93, #Bar
          20.27, #K
          32.98, #K
         [3.249,
         0.422e-3,
         0,
         0.083e5])

CO = Comp(
        0.045, # Prausnitz
        34.94, #Bar
        81.66, #K
        132.85, #K
        [ 3.376,
         0.557e-3,
         0,
         -0.031e5])

CO2 = Comp(
        0.225, # Prausnitz
        73.74, #Bar
        194.69, #K
        304.12, #K
        [ 5.457,
        1.045e-3,
        0,
        -1.157e5]) 

N2 = Comp(
        0.037,
        33.98,
        77.35,
        126.20,
        [3.280,
        0.593e-3,
        0,
        0.040e5])
    
CH3OH = Comp(
        0.565, # omega
        80.97, # Pc
        337.69, # Tb
        512.64, # Tc
        [2.211, # A
        12.216e-3, # B
        -3.450e-6, # C
        0], # D
        [5.15853, 1569.613, -34.846, 353.5, 512.63], # A, B, C, D, Tmin, Tmax Ambrose 1975
        [5.20409, 1581.341, -33.50, 288.1, 356.83], # Ambrose 1970
        [5.31301, 1676.569, -21.728, 353, 483]) # Hirata 1967

H2O = Comp(
        0.344, # Prausnitz
        220.64, #Bar
        373.15, #K
        647.14, #K
        [3.470, # V.Ness - |
        1.450e-3,
        0,
        0.121e5],
        [3.55959, 643.748, -198.043, 373, 573], # A, B, C, D, Tmin, Tmax do NIST liu e lindsey
        [4.6543, 1435.264, -64.848, 255.9, 373] # Stull, 1947
        )


In [195]:
T6 = toK(280)
P6 = 50
Hliq = ((CH3OH.Cp_L(Tpad) + CH3OH.Cp_L(CH3OH.TAntoine(P6))) / 2) * (CH3OH.TAntoine(P6) - Tpad)
Hgas = ((CH3OH.Cp_ig(CH3OH.TAntoine(50)) + CH3OH.Cp_ig(T6)) / 2) * (T6 - CH3OH.TAntoine(P6))
vap =(CH3OH.Hvap())
res = (CH3OH.Hres(50, T6))
print(f"medio {Hliq + Hgas + vap + res}\n")
Hliq = quad(CH3OH.Cp_L, Tpad, CH3OH.TAntoine(P6))[0]
HGas = quad(CH3OH.Cp_ig, CH3OH.TAntoine(P6), T6)[0]
print(f"agora {Hliq + Hgas + vap + res}\n")

CH3OH.Hmolar(50, T6)


medio 75134.6678353562

agora 68509.47509659793



68510.76399381839

In [196]:
# Para descoberta da temperatura Corrente 3
C2 = [
    0,
    0,
    265.6704877,
    60.1569231,
    1163.056436,
    15.03923078,
]  # Vazões CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5

T2 = toK(163)
P2 = 52
C12 = [
    6.24030213769374,
    2.41078129059267,
    213.428133248607,
    0,
    12251.7033530114,
    393.828789949421,
]  # Vazões CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5

T12 = toK(22)
P12 = 52
# C-2
C_2 = (
    C2[2] * CO.Hmolar(P2, T2)
    + C2[3] * CO2.Hmolar(P2, T2)
    + C2[4] * H2.Hmolar(P2, T2)
    + C2[5] * N2.Hmolar(P2, T2)
) # Entalpia da corrente
# C-12
C_12 = (
    C12[0] * CH3OH.Hmolar(P12, T12)
    + C12[1] * H2O.Hmolar(P12, T12)
    + C12[2] * CO.Hmolar(P12, T12)
    + C12[3] * CO2.Hmolar(P12, T12)
    + C12[4] * H2.Hmolar(P12, T12)
    + C12[5] * N2.Hmolar(P12, T12)
) # Entalpia da corrente
# print(f"{C_12} kJ / mol H")
# C3
C3 = [
    6.24030213769363,
    2.41078129059269,
    479.098620992281,
    60.1569231043921,
    13414.759788997,
    408.868020725519,
]  # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5
P3 = 52  # Estimativa
C_3 = C_2 + C_12
T3 = 0
for T in np.arange(T12, T2, 0.01):
    C3ite = (
        C3[0] * CH3OH.Hmolar(P3, T)
        + C3[1] * H2O.Hmolar(P3, T)
        + C3[2] * CO.Hmolar(P3, T)
        + C3[3] * CO2.Hmolar(P3, T)
        + C3[4] * H2.Hmolar(P3, T)
        + C3[5] * N2.Hmolar(P3, T)
    )
    # print(f"Dif:{abs(C_3 - C3ite)}, T:{T - 273.15}")
    if C3ite < C_3 + 1e4 and C3ite > C_3 - 1e4:
        print(T - 273.15)
        T3 = T
        break

# C-3
C_3 = ( 
    C3[0] * CH3OH.Hmolar(P3, T3)
    + C3[1] * H2O.Hmolar(P3, T3)
    + C3[2] * CO.Hmolar(P3, T3)
    + C3[3] * CO2.Hmolar(P3, T3)
    + C3[4] * H2.Hmolar(P3, T3)
    + C3[5] * N2.Hmolar(P3, T3)
)


37.03999999998632


In [197]:
# TC-1
# Tubos C3 -> C4
# C-4 C3 = C4 vazão igual
T4 = toK(220)
P4 = 51
C4 = C3
C_4 = (
    C4[0] * CH3OH.Hmolar(P4, T4)
    + C4[1] * H2O.Hmolar(P4, T4)
    + C4[2] * CO.Hmolar(P4, T4)
    + C4[3] * CO2.Hmolar(P4, T4)
    + C4[4] * H2.Hmolar(P4, T4)
    + C4[5] * N2.Hmolar(P4, T4)
) # Entalpia da corrente
print(f'C3:{C_3}, C4: {C_4}, Diff: {C_4 - C_3}')
QTC1 = C_4 - C_3

# Casco C6 -> C7
T6 = toK(280)
P6 = 50
C6 = [
    323.912734025437,
    62.5677043949848,
    221.58311220893,
    0,
    12719.2580021172,
    408.868020725519,
] # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5

C_6 = (
    C6[0] * CH3OH.Hmolar(P6, T6)
    + C6[1] * H2O.Hmolar(P6, T6)
    + C6[2] * CO.Hmolar(P6, T6)
    + C6[3] * CO2.Hmolar(P6, T6)
    + C6[4] * H2.Hmolar(P6, T6)
    + C6[5] * N2.Hmolar(P6, T6)
) # Entalpia da corrente
C7 = C6
P7 = P6

# Qtubos = -Qcasco
for T in np.arange(T3, T6, 0.01):
    C7ite = (
        C7[0] * CH3OH.Hmolar(P7, T)
        + C7[1] * H2O.Hmolar(P7, T)
        + C7[2] * CO.Hmolar(P7, T)
        + C7[3] * CO2.Hmolar(P7, T)
        + C7[4] * H2.Hmolar(P7, T)
        + C7[5] * N2.Hmolar(P7, T)
    )
    QTC1ite = C7ite - C_6 
    print(f"Diff:{abs(QTC1 + QTC1ite)}, T:{T - 273.15}")
    if QTC1 + QTC1ite < 1e3 and QTC1ite + QTC1 > -1e3:
        print(T - 273.15)
        T7 = T
        break
T = T4
C7ite = (
        C7[0] * CH3OH.Hmolar(P7, T)
        + C7[1] * H2O.Hmolar(P7, T)
        + C7[2] * CO.Hmolar(P7, T)
        + C7[3] * CO2.Hmolar(P7, T)
        + C7[4] * H2.Hmolar(P7, T)
        + C7[5] * N2.Hmolar(P7, T)
    )
print(C7ite)


C3:5616162.832848277, C4: 82630120.34226295, Diff: 77013957.50941467
Diff:42990383.82977368, T:37.03999999998632
Diff:42986016.021100074, T:37.04999999998631
Diff:42981648.217504874, T:37.0599999999863
Diff:42977280.4189868, T:37.069999999986294
Diff:42972912.62554443, T:37.079999999986285
Diff:42968544.83717641, T:37.089999999986276
Diff:42964177.05388141, T:37.09999999998627
Diff:42959809.27565798, T:37.10999999998626
Diff:42955441.502504855, T:37.11999999998625
Diff:42951073.73442063, T:37.12999999998624
Diff:42946705.97140396, T:37.13999999998623
Diff:42942338.21345353, T:37.14999999998622
Diff:42937970.46056791, T:37.15999999998621
Diff:42933602.7127457, T:37.1699999999862
Diff:42929234.96998569, T:37.179999999986194
Diff:42924867.232286304, T:37.189999999986185
Diff:42920499.49964647, T:37.199999999986176
Diff:42916131.7720646, T:37.20999999998617
Diff:42911764.04953945, T:37.21999999998616
Diff:42907396.33206959, T:37.22999999998615
Diff:42903028.61965366, T:37.23999999998614
Di

KeyboardInterrupt: 